In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -pe mthread 15
#$ -q sThC.q
#$ -l mres=120G,h_data=8G,h_vmem=8G
#$ -cwd
#$ -j y
#$ -N index_oyster
#$ -o index_oyster.log
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/bowtie2
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#
bowtie2-build GCA_002022765.4_C_virginica-3.0_genomic.fna C_virginica-3.0_genomic.fna_index --threads 15
#
echo = `date` job $JOB_NAME done

In [ ]:
| sed 's/[^-]*$//'

In [ ]:
# /bin/sh 
# ----------------Parameters---------------------- #
#$  -S /bin/sh
#$ -pe mthread 12
#$ -q mThC.q
#$ -l mres=72G,h_data=6G,h_vmem=6G
#$ -cwd
#$ -j y
#$ -N trim_ce
#$ -o trim_ce.log
#$ -m bea
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/trim_galore
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#
for f in $(ls | sed 's/[^-]*$//'); 
do 
    trim_galore -q 20 --phred33 --fastqc --illumina --length 20 --paired "$f"CV_R1_001.fastq "$f"CV_R2_001.fastq;
done

#
echo = `date` job $JOB_NAME done

Ok, so groves had these degenerate bases but we dont have that, and our adapters were already removed since they are just the standard illumina adapters. So I think just go straight to mapping adn then use picard tools to remove duplicates on bam files? 

In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -pe mthread 12
#$ -q mThC.q
#$ -l mres=72G,h_data=6G,h_vmem=6G
#$ -cwd
#$ -j y
#$ -N map_CE
#$ -o map_CE.log
#$ -m bea
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/bowtie2
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#


for f in $(ls | sed 's/[^-]*$//'); 
do 

bowtie2 -x /pool/genomics/gignoux-wolfsohns/Reference_genomes/cvirginica_genome/C_virginica-3.0_genomic.fna_index -1 /pool/genomics/gignoux-wolfsohns/CE_MR/trimmed_seqs/"$f"CV_R1_001_val_1.fq -2 /pool/genomics/gignoux-wolfsohns/CE_MR/trimmed_seqs/"$f"CV_R2_001_val_2.fq -p 12 -S /pool/genomics/gignoux-wolfsohns/CE_MR/mapped_sam/"$f".sam; 

done

#
echo = `date` job $JOB_NAME done


+ Thu Dec 16 14:48:18 EST 2021 job map_CE started in mThC.q with jobID=1722861 on compute-64-10
+ NSLOTS = 12
14563142 reads; of these:
  14563142 (100.00%) were paired; of these:
    11897170 (81.69%) aligned concordantly 0 times
    1302375 (8.94%) aligned concordantly exactly 1 time
    1363597 (9.36%) aligned concordantly >1 times
    ----
    11897170 pairs aligned concordantly 0 times; of these:
      440947 (3.71%) aligned discordantly 1 time
    ----
    11456223 pairs aligned 0 times concordantly or discordantly; of these:
      22912446 mates make up the pairs; of these:
        21159684 (92.35%) aligned 0 times
        567877 (2.48%) aligned exactly 1 time
        1184885 (5.17%) aligned >1 times
27.35% overall alignment rate
14563142 reads; of these:
  14563142 (100.00%) were paired; of these:
    11897170 (81.69%) aligned concordantly 0 times
    1302375 (8.94%) aligned concordantly exactly 1 time
    1363597 (9.36%) aligned concordantly >1 times
    ----
    11897170 pairs aligned concordantly 0 times; of these:
      440947 (3.71%) aligned discordantly 1 time
    ----
    11456223 pairs aligned 0 times concordantly or discordantly; of these:
      22912446 mates make up the pairs; of these:
        21159684 (92.35%) aligned 0 times
        567877 (2.48%) aligned exactly 1 time
        1184885 (5.17%) aligned >1 times
27.35% overall alignment rate
11404712 reads; of these:
  11404712 (100.00%) were paired; of these:
    9548763 (83.73%) aligned concordantly 0 times
    872191 (7.65%) aligned concordantly exactly 1 time
    983758 (8.63%) aligned concordantly >1 times
    ----
    9548763 pairs aligned concordantly 0 times; of these:
      313041 (3.28%) aligned discordantly 1 time
    ----
    9235722 pairs aligned 0 times concordantly or discordantly; of these:
      18471444 mates make up the pairs; of these:
        17244029 (93.36%) aligned 0 times
        371120 (2.01%) aligned exactly 1 time
        856295 (4.64%) aligned >1 times
24.40% overall alignment rate
11404712 reads; of these:
  11404712 (100.00%) were paired; of these:
    9548763 (83.73%) aligned concordantly 0 times
    872191 (7.65%) aligned concordantly exactly 1 time
    983758 (8.63%) aligned concordantly >1 times
    ----
    9548763 pairs aligned concordantly 0 times; of these:
      313041 (3.28%) aligned discordantly 1 time
    ----
    9235722 pairs aligned 0 times concordantly or discordantly; of these:
      18471444 mates make up the pairs; of these:
        17244029 (93.36%) aligned 0 times
        371120 (2.01%) aligned exactly 1 time
        856295 (4.64%) aligned >1 times
24.40% overall alignment rate
13128140 reads; of these:
  13128140 (100.00%) were paired; of these:
    10580226 (80.59%) aligned concordantly 0 times
    1276927 (9.73%) aligned concordantly exactly 1 time
    1270987 (9.68%) aligned concordantly >1 times
    ----
    10580226 pairs aligned concordantly 0 times; of these:
      407272 (3.85%) aligned discordantly 1 time
    ----
    10172954 pairs aligned 0 times concordantly or discordantly; of these:
      20345908 mates make up the pairs; of these:
        18654199 (91.69%) aligned 0 times
        557377 (2.74%) aligned exactly 1 time
        1134332 (5.58%) aligned >1 times
28.95% overall alignment rate
13128140 reads; of these:
  13128140 (100.00%) were paired; of these:
    10580226 (80.59%) aligned concordantly 0 times
    1276927 (9.73%) aligned concordantly exactly 1 time
    1270987 (9.68%) aligned concordantly >1 times
    ----
    10580226 pairs aligned concordantly 0 times; of these:
      407272 (3.85%) aligned discordantly 1 time
    ----
    10172954 pairs aligned 0 times concordantly or discordantly; of these:
      20345908 mates make up the pairs; of these:
        18654199 (91.69%) aligned 0 times
        557377 (2.74%) aligned exactly 1 time
        1134332 (5.58%) aligned >1 times
28.95% overall alignment rate
8607939 reads; of these:
  8607939 (100.00%) were paired; of these:
    7034820 (81.72%) aligned concordantly 0 times
    761905 (8.85%) aligned concordantly exactly 1 time
    811214 (9.42%) aligned concordantly >1 times
    ----
    7034820 pairs aligned concordantly 0 times; of these:
      264078 (3.75%) aligned discordantly 1 time
    ----
    6770742 pairs aligned 0 times concordantly or discordantly; of these:
      13541484 mates make up the pairs; of these:
        12490128 (92.24%) aligned 0 times
        324407 (2.40%) aligned exactly 1 time
        726949 (5.37%) aligned >1 times
27.45% overall alignment rate
8607939 reads; of these:
  8607939 (100.00%) were paired; of these:
    7034820 (81.72%) aligned concordantly 0 times
    761905 (8.85%) aligned concordantly exactly 1 time
    811214 (9.42%) aligned concordantly >1 times
    ----
    7034820 pairs aligned concordantly 0 times; of these:
      264078 (3.75%) aligned discordantly 1 time
    ----
    6770742 pairs aligned 0 times concordantly or discordantly; of these:
      13541484 mates make up the pairs; of these:
        12490128 (92.24%) aligned 0 times
        324407 (2.40%) aligned exactly 1 time
        726949 (5.37%) aligned >1 times
27.45% overall alignment rate
14556378 reads; of these:
  14556378 (100.00%) were paired; of these:
    12118213 (83.25%) aligned concordantly 0 times
    1175509 (8.08%) aligned concordantly exactly 1 time
    1262656 (8.67%) aligned concordantly >1 times
    ----
    12118213 pairs aligned concordantly 0 times; of these:
      414216 (3.42%) aligned discordantly 1 time
    ----
    11703997 pairs aligned 0 times concordantly or discordantly; of these:
      23407994 mates make up the pairs; of these:
        21775063 (93.02%) aligned 0 times
        511386 (2.18%) aligned exactly 1 time
        1121545 (4.79%) aligned >1 times
25.20% overall alignment rate
14556378 reads; of these:
  14556378 (100.00%) were paired; of these:
    12118213 (83.25%) aligned concordantly 0 times
    1175509 (8.08%) aligned concordantly exactly 1 time
    1262656 (8.67%) aligned concordantly >1 times
    ----
    12118213 pairs aligned concordantly 0 times; of these:
      414216 (3.42%) aligned discordantly 1 time
    ----
    11703997 pairs aligned 0 times concordantly or discordantly; of these:
      23407994 mates make up the pairs; of these:
        21775063 (93.02%) aligned 0 times
        511386 (2.18%) aligned exactly 1 time
        1121545 (4.79%) aligned >1 times
25.20% overall alignment rate
12093094 reads; of these:
  12093094 (100.00%) were paired; of these:
    10192069 (84.28%) aligned concordantly 0 times
    884865 (7.32%) aligned concordantly exactly 1 time
    1016160 (8.40%) aligned concordantly >1 times
    ----
    10192069 pairs aligned concordantly 0 times; of these:
      326196 (3.20%) aligned discordantly 1 time
    ----
    9865873 pairs aligned 0 times concordantly or discordantly; of these:
      19731746 mates make up the pairs; of these:
        18477271 (93.64%) aligned 0 times
        365555 (1.85%) aligned exactly 1 time
        888920 (4.51%) aligned >1 times
23.60% overall alignment rate
12093094 reads; of these:
  12093094 (100.00%) were paired; of these:
    10192069 (84.28%) aligned concordantly 0 times
    884865 (7.32%) aligned concordantly exactly 1 time
    1016160 (8.40%) aligned concordantly >1 times
    ----
    10192069 pairs aligned concordantly 0 times; of these:
      326196 (3.20%) aligned discordantly 1 time
    ----
    9865873 pairs aligned 0 times concordantly or discordantly; of these:
      19731746 mates make up the pairs; of these:
        18477271 (93.64%) aligned 0 times
        365555 (1.85%) aligned exactly 1 time
        888920 (4.51%) aligned >1 times
23.60% overall alignment rate
7993293 reads; of these:
  7993293 (100.00%) were paired; of these:
    6648490 (83.18%) aligned concordantly 0 times
    669354 (8.37%) aligned concordantly exactly 1 time
    675449 (8.45%) aligned concordantly >1 times
    ----
    6648490 pairs aligned concordantly 0 times; of these:
      237576 (3.57%) aligned discordantly 1 time
    ----
    6410914 pairs aligned 0 times concordantly or discordantly; of these:
      12821828 mates make up the pairs; of these:
        11845029 (92.38%) aligned 0 times
        312134 (2.43%) aligned exactly 1 time
        664665 (5.18%) aligned >1 times
25.91% overall alignment rate
7993293 reads; of these:
  7993293 (100.00%) were paired; of these:
    6648490 (83.18%) aligned concordantly 0 times
    669354 (8.37%) aligned concordantly exactly 1 time
    675449 (8.45%) aligned concordantly >1 times
    ----
    6648490 pairs aligned concordantly 0 times; of these:
      237576 (3.57%) aligned discordantly 1 time
    ----
    6410914 pairs aligned 0 times concordantly or discordantly; of these:
      12821828 mates make up the pairs; of these:
        11845029 (92.38%) aligned 0 times
        312134 (2.43%) aligned exactly 1 time
        664665 (5.18%) aligned >1 times
25.91% overall alignment rate
11909936 reads; of these:
  11909936 (100.00%) were paired; of these:
    9825028 (82.49%) aligned concordantly 0 times
    1024372 (8.60%) aligned concordantly exactly 1 time
    1060536 (8.90%) aligned concordantly >1 times
    ----
    9825028 pairs aligned concordantly 0 times; of these:
      349511 (3.56%) aligned discordantly 1 time
    ----
    9475517 pairs aligned 0 times concordantly or discordantly; of these:
      18951034 mates make up the pairs; of these:
        17584135 (92.79%) aligned 0 times
        427540 (2.26%) aligned exactly 1 time
        939359 (4.96%) aligned >1 times
26.18% overall alignment rate
11909936 reads; of these:
  11909936 (100.00%) were paired; of these:
    9825028 (82.49%) aligned concordantly 0 times
    1024372 (8.60%) aligned concordantly exactly 1 time
    1060536 (8.90%) aligned concordantly >1 times
    ----
    9825028 pairs aligned concordantly 0 times; of these:
      349511 (3.56%) aligned discordantly 1 time
    ----
    9475517 pairs aligned 0 times concordantly or discordantly; of these:
      18951034 mates make up the pairs; of these:
        17584135 (92.79%) aligned 0 times
        427540 (2.26%) aligned exactly 1 time
        939359 (4.96%) aligned >1 times
26.18% overall alignment rate
14797409 reads; of these:
  14797409 (100.00%) were paired; of these:
    11778758 (79.60%) aligned concordantly 0 times
    1473596 (9.96%) aligned concordantly exactly 1 time
    1545055 (10.44%) aligned concordantly >1 times
    ----
    11778758 pairs aligned concordantly 0 times; of these:
      472201 (4.01%) aligned discordantly 1 time
    ----
    11306557 pairs aligned 0 times concordantly or discordantly; of these:
      22613114 mates make up the pairs; of these:
        20702654 (91.55%) aligned 0 times
        596119 (2.64%) aligned exactly 1 time
        1314341 (5.81%) aligned >1 times
30.05% overall alignment rate
14797409 reads; of these:
  14797409 (100.00%) were paired; of these:
    11778758 (79.60%) aligned concordantly 0 times
    1473596 (9.96%) aligned concordantly exactly 1 time
    1545055 (10.44%) aligned concordantly >1 times
    ----
    11778758 pairs aligned concordantly 0 times; of these:
      472201 (4.01%) aligned discordantly 1 time
    ----
    11306557 pairs aligned 0 times concordantly or discordantly; of these:
      22613114 mates make up the pairs; of these:
        20702654 (91.55%) aligned 0 times
        596119 (2.64%) aligned exactly 1 time
        1314341 (5.81%) aligned >1 times
30.05% overall alignment rate
5736254 reads; of these:
  5736254 (100.00%) were paired; of these:
    4839900 (84.37%) aligned concordantly 0 times
    413219 (7.20%) aligned concordantly exactly 1 time
    483135 (8.42%) aligned concordantly >1 times
    ----
    4839900 pairs aligned concordantly 0 times; of these:
      157293 (3.25%) aligned discordantly 1 time
    ----
    4682607 pairs aligned 0 times concordantly or discordantly; of these:
      9365214 mates make up the pairs; of these:
        8678339 (92.67%) aligned 0 times
        214571 (2.29%) aligned exactly 1 time
        472304 (5.04%) aligned >1 times
24.36% overall alignment rate
5736254 reads; of these:
  5736254 (100.00%) were paired; of these:
    4839900 (84.37%) aligned concordantly 0 times
    413219 (7.20%) aligned concordantly exactly 1 time
    483135 (8.42%) aligned concordantly >1 times
    ----
    4839900 pairs aligned concordantly 0 times; of these:
      157293 (3.25%) aligned discordantly 1 time
    ----
    4682607 pairs aligned 0 times concordantly or discordantly; of these:
      9365214 mates make up the pairs; of these:
        8678339 (92.67%) aligned 0 times
        214571 (2.29%) aligned exactly 1 time
        472304 (5.04%) aligned >1 times
24.36% overall alignment rate
5090373 reads; of these:
  5090373 (100.00%) were paired; of these:
    4307128 (84.61%) aligned concordantly 0 times
    374173 (7.35%) aligned concordantly exactly 1 time
    409072 (8.04%) aligned concordantly >1 times
    ----
    4307128 pairs aligned concordantly 0 times; of these:
      137876 (3.20%) aligned discordantly 1 time
    ----
    4169252 pairs aligned 0 times concordantly or discordantly; of these:
      8338504 mates make up the pairs; of these:
        7700678 (92.35%) aligned 0 times
        206654 (2.48%) aligned exactly 1 time
        431172 (5.17%) aligned >1 times
24.36% overall alignment rate
5090373 reads; of these:
  5090373 (100.00%) were paired; of these:
    4307128 (84.61%) aligned concordantly 0 times
    374173 (7.35%) aligned concordantly exactly 1 time
    409072 (8.04%) aligned concordantly >1 times
    ----
    4307128 pairs aligned concordantly 0 times; of these:
      137876 (3.20%) aligned discordantly 1 time
    ----
    4169252 pairs aligned 0 times concordantly or discordantly; of these:
      8338504 mates make up the pairs; of these:
        7700678 (92.35%) aligned 0 times
        206654 (2.48%) aligned exactly 1 time
        431172 (5.17%) aligned >1 times
24.36% overall alignment rate
3816318 reads; of these:
  3816318 (100.00%) were paired; of these:
    3220015 (84.37%) aligned concordantly 0 times
    304669 (7.98%) aligned concordantly exactly 1 time
    291634 (7.64%) aligned concordantly >1 times
    ----
    3220015 pairs aligned concordantly 0 times; of these:
      110416 (3.43%) aligned discordantly 1 time
    ----
    3109599 pairs aligned 0 times concordantly or discordantly; of these:
      6219198 mates make up the pairs; of these:
        5754289 (92.52%) aligned 0 times
        150406 (2.42%) aligned exactly 1 time
        314503 (5.06%) aligned >1 times
24.61% overall alignment rate
3816318 reads; of these:
  3816318 (100.00%) were paired; of these:
    3220015 (84.37%) aligned concordantly 0 times
    304669 (7.98%) aligned concordantly exactly 1 time
    291634 (7.64%) aligned concordantly >1 times
    ----
    3220015 pairs aligned concordantly 0 times; of these:
      110416 (3.43%) aligned discordantly 1 time
    ----
    3109599 pairs aligned 0 times concordantly or discordantly; of these:
      6219198 mates make up the pairs; of these:
        5754289 (92.52%) aligned 0 times
        150406 (2.42%) aligned exactly 1 time
        314503 (5.06%) aligned >1 times
24.61% overall alignment rate
9241072 reads; of these:
  9241072 (100.00%) were paired; of these:
    7392486 (80.00%) aligned concordantly 0 times
    924318 (10.00%) aligned concordantly exactly 1 time
    924268 (10.00%) aligned concordantly >1 times
    ----
    7392486 pairs aligned concordantly 0 times; of these:
      306934 (4.15%) aligned discordantly 1 time
    ----
    7085552 pairs aligned 0 times concordantly or discordantly; of these:
      14171104 mates make up the pairs; of these:
        12918735 (91.16%) aligned 0 times
        412376 (2.91%) aligned exactly 1 time
        839993 (5.93%) aligned >1 times
30.10% overall alignment rate
9241072 reads; of these:
  9241072 (100.00%) were paired; of these:
    7392486 (80.00%) aligned concordantly 0 times
    924318 (10.00%) aligned concordantly exactly 1 time
    924268 (10.00%) aligned concordantly >1 times
    ----
    7392486 pairs aligned concordantly 0 times; of these:
      306934 (4.15%) aligned discordantly 1 time
    ----
    7085552 pairs aligned 0 times concordantly or discordantly; of these:
      14171104 mates make up the pairs; of these:
        12918735 (91.16%) aligned 0 times
        412376 (2.91%) aligned exactly 1 time
        839993 (5.93%) aligned >1 times
30.10% overall alignment rate
4056137 reads; of these:
  4056137 (100.00%) were paired; of these:
    3435881 (84.71%) aligned concordantly 0 times
    310276 (7.65%) aligned concordantly exactly 1 time
    309980 (7.64%) aligned concordantly >1 times
    ----
    3435881 pairs aligned concordantly 0 times; of these:
      116306 (3.39%) aligned discordantly 1 time
    ----
    3319575 pairs aligned 0 times concordantly or discordantly; of these:
      6639150 mates make up the pairs; of these:
        6183726 (93.14%) aligned 0 times
        139185 (2.10%) aligned exactly 1 time
        316239 (4.76%) aligned >1 times
23.77% overall alignment rate
4056137 reads; of these:
  4056137 (100.00%) were paired; of these:
    3435881 (84.71%) aligned concordantly 0 times
    310276 (7.65%) aligned concordantly exactly 1 time
    309980 (7.64%) aligned concordantly >1 times
    ----
    3435881 pairs aligned concordantly 0 times; of these:
      116306 (3.39%) aligned discordantly 1 time
    ----
    3319575 pairs aligned 0 times concordantly or discordantly; of these:
      6639150 mates make up the pairs; of these:
        6183726 (93.14%) aligned 0 times
        139185 (2.10%) aligned exactly 1 time
        316239 (4.76%) aligned >1 times
23.77% overall alignment rate
3106914 reads; of these:
  3106914 (100.00%) were paired; of these:
    2570143 (82.72%) aligned concordantly 0 times
    260726 (8.39%) aligned concordantly exactly 1 time
    276045 (8.88%) aligned concordantly >1 times
    ----
    2570143 pairs aligned concordantly 0 times; of these:
      96429 (3.75%) aligned discordantly 1 time
    ----
    2473714 pairs aligned 0 times concordantly or discordantly; of these:
      4947428 mates make up the pairs; of these:
        4513535 (91.23%) aligned 0 times
        137874 (2.79%) aligned exactly 1 time
        296019 (5.98%) aligned >1 times
27.36% overall alignment rate
3106914 reads; of these:
  3106914 (100.00%) were paired; of these:
    2570143 (82.72%) aligned concordantly 0 times
    260726 (8.39%) aligned concordantly exactly 1 time
    276045 (8.88%) aligned concordantly >1 times
    ----
    2570143 pairs aligned concordantly 0 times; of these:
      96429 (3.75%) aligned discordantly 1 time
    ----
    2473714 pairs aligned 0 times concordantly or discordantly; of these:
      4947428 mates make up the pairs; of these:
        4513535 (91.23%) aligned 0 times
        137874 (2.79%) aligned exactly 1 time
        296019 (5.98%) aligned >1 times
27.36% overall alignment rate
4341135 reads; of these:
  4341135 (100.00%) were paired; of these:
    3607167 (83.09%) aligned concordantly 0 times
    366883 (8.45%) aligned concordantly exactly 1 time
    367085 (8.46%) aligned concordantly >1 times
    ----
    3607167 pairs aligned concordantly 0 times; of these:
      126335 (3.50%) aligned discordantly 1 time
    ----
    3480832 pairs aligned 0 times concordantly or discordantly; of these:
      6961664 mates make up the pairs; of these:
        6434699 (92.43%) aligned 0 times
        168827 (2.43%) aligned exactly 1 time
        358138 (5.14%) aligned >1 times
25.89% overall alignment rate
4341135 reads; of these:
  4341135 (100.00%) were paired; of these:
    3607167 (83.09%) aligned concordantly 0 times
    366883 (8.45%) aligned concordantly exactly 1 time
    367085 (8.46%) aligned concordantly >1 times
    ----
    3607167 pairs aligned concordantly 0 times; of these:
      126335 (3.50%) aligned discordantly 1 time
    ----
    3480832 pairs aligned 0 times concordantly or discordantly; of these:
      6961664 mates make up the pairs; of these:
        6434699 (92.43%) aligned 0 times
        168827 (2.43%) aligned exactly 1 time
        358138 (5.14%) aligned >1 times
25.89% overall alignment rate
10029339 reads; of these:
  10029339 (100.00%) were paired; of these:
    9230711 (92.04%) aligned concordantly 0 times
    368193 (3.67%) aligned concordantly exactly 1 time
    430435 (4.29%) aligned concordantly >1 times
    ----
    9230711 pairs aligned concordantly 0 times; of these:
      190271 (2.06%) aligned discordantly 1 time
    ----
    9040440 pairs aligned 0 times concordantly or discordantly; of these:
      18080880 mates make up the pairs; of these:
        17354160 (95.98%) aligned 0 times
        196728 (1.09%) aligned exactly 1 time
        529992 (2.93%) aligned >1 times
13.48% overall alignment rate
10029339 reads; of these:
  10029339 (100.00%) were paired; of these:
    9230711 (92.04%) aligned concordantly 0 times
    368193 (3.67%) aligned concordantly exactly 1 time
    430435 (4.29%) aligned concordantly >1 times
    ----
    9230711 pairs aligned concordantly 0 times; of these:
      190271 (2.06%) aligned discordantly 1 time
    ----
    9040440 pairs aligned 0 times concordantly or discordantly; of these:
      18080880 mates make up the pairs; of these:
        17354160 (95.98%) aligned 0 times
        196728 (1.09%) aligned exactly 1 time
        529992 (2.93%) aligned >1 times
13.48% overall alignment rate
14641056 reads; of these:
  14641056 (100.00%) were paired; of these:
    12794656 (87.39%) aligned concordantly 0 times
    854871 (5.84%) aligned concordantly exactly 1 time
    991529 (6.77%) aligned concordantly >1 times
    ----
    12794656 pairs aligned concordantly 0 times; of these:
      419212 (3.28%) aligned discordantly 1 time
    ----
    12375444 pairs aligned 0 times concordantly or discordantly; of these:
      24750888 mates make up the pairs; of these:
        23285174 (94.08%) aligned 0 times
        395781 (1.60%) aligned exactly 1 time
        1069933 (4.32%) aligned >1 times
20.48% overall alignment rate
14641056 reads; of these:
  14641056 (100.00%) were paired; of these:
    12794656 (87.39%) aligned concordantly 0 times
    854871 (5.84%) aligned concordantly exactly 1 time
    991529 (6.77%) aligned concordantly >1 times
    ----
    12794656 pairs aligned concordantly 0 times; of these:
      419212 (3.28%) aligned discordantly 1 time
    ----
    12375444 pairs aligned 0 times concordantly or discordantly; of these:
      24750888 mates make up the pairs; of these:
        23285174 (94.08%) aligned 0 times
        395781 (1.60%) aligned exactly 1 time
        1069933 (4.32%) aligned >1 times
20.48% overall alignment rate
3758474 reads; of these:
  3758474 (100.00%) were paired; of these:
    3167534 (84.28%) aligned concordantly 0 times
    280568 (7.46%) aligned concordantly exactly 1 time
    310372 (8.26%) aligned concordantly >1 times
    ----
    3167534 pairs aligned concordantly 0 times; of these:
      102676 (3.24%) aligned discordantly 1 time
    ----
    3064858 pairs aligned 0 times concordantly or discordantly; of these:
      6129716 mates make up the pairs; of these:
        5673731 (92.56%) aligned 0 times
        142797 (2.33%) aligned exactly 1 time
        313188 (5.11%) aligned >1 times
24.52% overall alignment rate
3758474 reads; of these:
  3758474 (100.00%) were paired; of these:
    3167534 (84.28%) aligned concordantly 0 times
    280568 (7.46%) aligned concordantly exactly 1 time
    310372 (8.26%) aligned concordantly >1 times
    ----
    3167534 pairs aligned concordantly 0 times; of these:
      102676 (3.24%) aligned discordantly 1 time
    ----
    3064858 pairs aligned 0 times concordantly or discordantly; of these:
      6129716 mates make up the pairs; of these:
        5673731 (92.56%) aligned 0 times
        142797 (2.33%) aligned exactly 1 time
        313188 (5.11%) aligned >1 times
24.52% overall alignment rate
1891962 reads; of these:
  1891962 (100.00%) were paired; of these:
    1684044 (89.01%) aligned concordantly 0 times
    82585 (4.37%) aligned concordantly exactly 1 time
    125333 (6.62%) aligned concordantly >1 times
    ----
    1684044 pairs aligned concordantly 0 times; of these:
      35011 (2.08%) aligned discordantly 1 time
    ----
    1649033 pairs aligned 0 times concordantly or discordantly; of these:
      3298066 mates make up the pairs; of these:
        2964742 (89.89%) aligned 0 times
        117097 (3.55%) aligned exactly 1 time
        216227 (6.56%) aligned >1 times
21.65% overall alignment rate
1891962 reads; of these:
  1891962 (100.00%) were paired; of these:
    1684044 (89.01%) aligned concordantly 0 times
    82585 (4.37%) aligned concordantly exactly 1 time
    125333 (6.62%) aligned concordantly >1 times
    ----
    1684044 pairs aligned concordantly 0 times; of these:
      35011 (2.08%) aligned discordantly 1 time
    ----
    1649033 pairs aligned 0 times concordantly or discordantly; of these:
      3298066 mates make up the pairs; of these:
        2964742 (89.89%) aligned 0 times
        117097 (3.55%) aligned exactly 1 time
        216227 (6.56%) aligned >1 times
21.65% overall alignment rate
9703101 reads; of these:
  9703101 (100.00%) were paired; of these:
    8723771 (89.91%) aligned concordantly 0 times
    456383 (4.70%) aligned concordantly exactly 1 time
    522947 (5.39%) aligned concordantly >1 times
    ----
    8723771 pairs aligned concordantly 0 times; of these:
      219103 (2.51%) aligned discordantly 1 time
    ----
    8504668 pairs aligned 0 times concordantly or discordantly; of these:
      17009336 mates make up the pairs; of these:
        16216971 (95.34%) aligned 0 times
        218778 (1.29%) aligned exactly 1 time
        573587 (3.37%) aligned >1 times
16.43% overall alignment rate
9703101 reads; of these:
  9703101 (100.00%) were paired; of these:
    8723771 (89.91%) aligned concordantly 0 times
    456383 (4.70%) aligned concordantly exactly 1 time
    522947 (5.39%) aligned concordantly >1 times
    ----
    8723771 pairs aligned concordantly 0 times; of these:
      219103 (2.51%) aligned discordantly 1 time
    ----
    8504668 pairs aligned 0 times concordantly or discordantly; of these:
      17009336 mates make up the pairs; of these:
        16216971 (95.34%) aligned 0 times
        218778 (1.29%) aligned exactly 1 time
        573587 (3.37%) aligned >1 times
16.43% overall alignment rate
4755843 reads; of these:
  4755843 (100.00%) were paired; of these:
    4120432 (86.64%) aligned concordantly 0 times
    303954 (6.39%) aligned concordantly exactly 1 time
    331457 (6.97%) aligned concordantly >1 times
    ----
    4120432 pairs aligned concordantly 0 times; of these:
      129726 (3.15%) aligned discordantly 1 time
    ----
    3990706 pairs aligned 0 times concordantly or discordantly; of these:
      7981412 mates make up the pairs; of these:
        7486631 (93.80%) aligned 0 times
        142755 (1.79%) aligned exactly 1 time
        352026 (4.41%) aligned >1 times
21.29% overall alignment rate
4755843 reads; of these:
  4755843 (100.00%) were paired; of these:
    4120432 (86.64%) aligned concordantly 0 times
    303954 (6.39%) aligned concordantly exactly 1 time
    331457 (6.97%) aligned concordantly >1 times
    ----
    4120432 pairs aligned concordantly 0 times; of these:
      129726 (3.15%) aligned discordantly 1 time
    ----
    3990706 pairs aligned 0 times concordantly or discordantly; of these:
      7981412 mates make up the pairs; of these:
        7486631 (93.80%) aligned 0 times
        142755 (1.79%) aligned exactly 1 time
        352026 (4.41%) aligned >1 times
21.29% overall alignment rate
2568685 reads; of these:
  2568685 (100.00%) were paired; of these:
    2144239 (83.48%) aligned concordantly 0 times
    199009 (7.75%) aligned concordantly exactly 1 time
    225437 (8.78%) aligned concordantly >1 times
    ----
    2144239 pairs aligned concordantly 0 times; of these:
      71617 (3.34%) aligned discordantly 1 time
    ----
    2072622 pairs aligned 0 times concordantly or discordantly; of these:
      4145244 mates make up the pairs; of these:
        3781530 (91.23%) aligned 0 times
        122488 (2.95%) aligned exactly 1 time
        241226 (5.82%) aligned >1 times
26.39% overall alignment rate
2568685 reads; of these:
  2568685 (100.00%) were paired; of these:
    2144239 (83.48%) aligned concordantly 0 times
    199009 (7.75%) aligned concordantly exactly 1 time
    225437 (8.78%) aligned concordantly >1 times
    ----
    2144239 pairs aligned concordantly 0 times; of these:
      71617 (3.34%) aligned discordantly 1 time
    ----
    2072622 pairs aligned 0 times concordantly or discordantly; of these:
      4145244 mates make up the pairs; of these:
        3781530 (91.23%) aligned 0 times
        122488 (2.95%) aligned exactly 1 time
        241226 (5.82%) aligned >1 times
26.39% overall alignment rate
12961849 reads; of these:
  12961849 (100.00%) were paired; of these:
    11815685 (91.16%) aligned concordantly 0 times
    539511 (4.16%) aligned concordantly exactly 1 time
    606653 (4.68%) aligned concordantly >1 times
    ----
    11815685 pairs aligned concordantly 0 times; of these:
      298337 (2.52%) aligned discordantly 1 time
    ----
    11517348 pairs aligned 0 times concordantly or discordantly; of these:
      23034696 mates make up the pairs; of these:
        21888278 (95.02%) aligned 0 times
        330999 (1.44%) aligned exactly 1 time
        815419 (3.54%) aligned >1 times
15.57% overall alignment rate
12961849 reads; of these:
  12961849 (100.00%) were paired; of these:
    11815685 (91.16%) aligned concordantly 0 times
    539511 (4.16%) aligned concordantly exactly 1 time
    606653 (4.68%) aligned concordantly >1 times
    ----
    11815685 pairs aligned concordantly 0 times; of these:
      298337 (2.52%) aligned discordantly 1 time
    ----
    11517348 pairs aligned 0 times concordantly or discordantly; of these:
      23034696 mates make up the pairs; of these:
        21888278 (95.02%) aligned 0 times
        330999 (1.44%) aligned exactly 1 time
        815419 (3.54%) aligned >1 times
15.57% overall alignment rate
6814756 reads; of these:
  6814756 (100.00%) were paired; of these:
    5916320 (86.82%) aligned concordantly 0 times
    420530 (6.17%) aligned concordantly exactly 1 time
    477906 (7.01%) aligned concordantly >1 times
    ----
    5916320 pairs aligned concordantly 0 times; of these:
      179436 (3.03%) aligned discordantly 1 time
    ----
    5736884 pairs aligned 0 times concordantly or discordantly; of these:
      11473768 mates make up the pairs; of these:
        10771935 (93.88%) aligned 0 times
        206949 (1.80%) aligned exactly 1 time
        494884 (4.31%) aligned >1 times
20.97% overall alignment rate
6814756 reads; of these:
  6814756 (100.00%) were paired; of these:
    5916320 (86.82%) aligned concordantly 0 times
    420530 (6.17%) aligned concordantly exactly 1 time
    477906 (7.01%) aligned concordantly >1 times
    ----
    5916320 pairs aligned concordantly 0 times; of these:
      179436 (3.03%) aligned discordantly 1 time
    ----
    5736884 pairs aligned 0 times concordantly or discordantly; of these:
      11473768 mates make up the pairs; of these:
        10771935 (93.88%) aligned 0 times
        206949 (1.80%) aligned exactly 1 time
        494884 (4.31%) aligned >1 times
20.97% overall alignment rate
8499948 reads; of these:
  8499948 (100.00%) were paired; of these:
    7413397 (87.22%) aligned concordantly 0 times
    531361 (6.25%) aligned concordantly exactly 1 time
    555190 (6.53%) aligned concordantly >1 times
    ----
    7413397 pairs aligned concordantly 0 times; of these:
      226618 (3.06%) aligned discordantly 1 time
    ----
    7186779 pairs aligned 0 times concordantly or discordantly; of these:
      14373558 mates make up the pairs; of these:
        13510349 (93.99%) aligned 0 times
        258793 (1.80%) aligned exactly 1 time
        604416 (4.21%) aligned >1 times
20.53% overall alignment rate
8499948 reads; of these:
  8499948 (100.00%) were paired; of these:
    7413397 (87.22%) aligned concordantly 0 times
    531361 (6.25%) aligned concordantly exactly 1 time
    555190 (6.53%) aligned concordantly >1 times
    ----
    7413397 pairs aligned concordantly 0 times; of these:
      226618 (3.06%) aligned discordantly 1 time
    ----
    7186779 pairs aligned 0 times concordantly or discordantly; of these:
      14373558 mates make up the pairs; of these:
        13510349 (93.99%) aligned 0 times
        258793 (1.80%) aligned exactly 1 time
        604416 (4.21%) aligned >1 times
20.53% overall alignment rate
6502981 reads; of these:
  6502981 (100.00%) were paired; of these:
    5187300 (79.77%) aligned concordantly 0 times
    671365 (10.32%) aligned concordantly exactly 1 time
    644316 (9.91%) aligned concordantly >1 times
    ----
    5187300 pairs aligned concordantly 0 times; of these:
      203501 (3.92%) aligned discordantly 1 time
    ----
    4983799 pairs aligned 0 times concordantly or discordantly; of these:
      9967598 mates make up the pairs; of these:
        8852161 (88.81%) aligned 0 times
        411317 (4.13%) aligned exactly 1 time
        704120 (7.06%) aligned >1 times
31.94% overall alignment rate
6502981 reads; of these:
  6502981 (100.00%) were paired; of these:
    5187300 (79.77%) aligned concordantly 0 times
    671365 (10.32%) aligned concordantly exactly 1 time
    644316 (9.91%) aligned concordantly >1 times
    ----
    5187300 pairs aligned concordantly 0 times; of these:
      203501 (3.92%) aligned discordantly 1 time
    ----
    4983799 pairs aligned 0 times concordantly or discordantly; of these:
      9967598 mates make up the pairs; of these:
        8852161 (88.81%) aligned 0 times
        411317 (4.13%) aligned exactly 1 time
        704120 (7.06%) aligned >1 times
31.94% overall alignment rate
5690432 reads; of these:
  5690432 (100.00%) were paired; of these:
    4613753 (81.08%) aligned concordantly 0 times
    471329 (8.28%) aligned concordantly exactly 1 time
    605350 (10.64%) aligned concordantly >1 times
    ----
    4613753 pairs aligned concordantly 0 times; of these:
      154471 (3.35%) aligned discordantly 1 time
    ----
    4459282 pairs aligned 0 times concordantly or discordantly; of these:
      8918564 mates make up the pairs; of these:
        8196444 (91.90%) aligned 0 times
        219433 (2.46%) aligned exactly 1 time
        502687 (5.64%) aligned >1 times
27.98% overall alignment rate
5690432 reads; of these:
  5690432 (100.00%) were paired; of these:
    4613753 (81.08%) aligned concordantly 0 times
    471329 (8.28%) aligned concordantly exactly 1 time
    605350 (10.64%) aligned concordantly >1 times
    ----
    4613753 pairs aligned concordantly 0 times; of these:
      154471 (3.35%) aligned discordantly 1 time
    ----
    4459282 pairs aligned 0 times concordantly or discordantly; of these:
      8918564 mates make up the pairs; of these:
        8196444 (91.90%) aligned 0 times
        219433 (2.46%) aligned exactly 1 time
        502687 (5.64%) aligned >1 times
27.98% overall alignment rate
6322964 reads; of these:
  6322964 (100.00%) were paired; of these:
    5494694 (86.90%) aligned concordantly 0 times
    371497 (5.88%) aligned concordantly exactly 1 time
    456773 (7.22%) aligned concordantly >1 times
    ----
    5494694 pairs aligned concordantly 0 times; of these:
      170929 (3.11%) aligned discordantly 1 time
    ----
    5323765 pairs aligned 0 times concordantly or discordantly; of these:
      10647530 mates make up the pairs; of these:
        9995587 (93.88%) aligned 0 times
        184921 (1.74%) aligned exactly 1 time
        467022 (4.39%) aligned >1 times
20.96% overall alignment rate
6322964 reads; of these:
  6322964 (100.00%) were paired; of these:
    5494694 (86.90%) aligned concordantly 0 times
    371497 (5.88%) aligned concordantly exactly 1 time
    456773 (7.22%) aligned concordantly >1 times
    ----
    5494694 pairs aligned concordantly 0 times; of these:
      170929 (3.11%) aligned discordantly 1 time
    ----
    5323765 pairs aligned 0 times concordantly or discordantly; of these:
      10647530 mates make up the pairs; of these:
        9995587 (93.88%) aligned 0 times
        184921 (1.74%) aligned exactly 1 time
        467022 (4.39%) aligned >1 times
20.96% overall alignment rate
5282540 reads; of these:
  5282540 (100.00%) were paired; of these:
    4453934 (84.31%) aligned concordantly 0 times
    397458 (7.52%) aligned concordantly exactly 1 time
    431148 (8.16%) aligned concordantly >1 times
    ----
    4453934 pairs aligned concordantly 0 times; of these:
      129711 (2.91%) aligned discordantly 1 time
    ----
    4324223 pairs aligned 0 times concordantly or discordantly; of these:
      8648446 mates make up the pairs; of these:
        7877819 (91.09%) aligned 0 times
        283331 (3.28%) aligned exactly 1 time
        487296 (5.63%) aligned >1 times
25.44% overall alignment rate
5282540 reads; of these:
  5282540 (100.00%) were paired; of these:
    4453934 (84.31%) aligned concordantly 0 times
    397458 (7.52%) aligned concordantly exactly 1 time
    431148 (8.16%) aligned concordantly >1 times
    ----
    4453934 pairs aligned concordantly 0 times; of these:
      129711 (2.91%) aligned discordantly 1 time
    ----
    4324223 pairs aligned 0 times concordantly or discordantly; of these:
      8648446 mates make up the pairs; of these:
        7877819 (91.09%) aligned 0 times
        283331 (3.28%) aligned exactly 1 time
        487296 (5.63%) aligned >1 times
25.44% overall alignment rate
= Thu Dec 16 20:09:21 EST 2021 job map_CE done


Trying with --very-sensitive --local, should give higher mapping percentages. 